In [14]:
import pandas as pd
import numpy as np

In [15]:
hotelreviews = pd.read_csv('/home/jack/code/TechLah/RevuSum/raw_data/Hotel_Reviews.csv')

In [16]:
df = hotelreviews.copy()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [18]:
df.head(3)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968


In [19]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


#Using preprocessing from previous assignment

stop_words = set(stopwords.words('english')) # you can also choose other languages

def preprocessing_stopwords(sentence):
    sentence = sentence.strip("") #whitespace
    sentence = sentence.lower() #lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) #remove number
    for punctuation in string.punctuation: #remove punc
        sentence = sentence.replace(punctuation, '')
    word_tokens = word_tokenize(sentence)
    
    stopwords_removed = [w for w in word_tokens if not w in stop_words] 
        # Lemmatizing the verbs
    verb_lemmatized = [                  
        WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
        for word in stopwords_removed]

    # 2 - Lemmatizing the nouns
    noun_lemmatized = [                 
        WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
        for word in verb_lemmatized]
    
    return noun_lemmatized


def preprocessing(sentence):
    sentence = sentence.strip("") #whitespace
    sentence = sentence.lower() #lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) #remove number
    for punctuation in string.punctuation: #remove punc
        sentence = sentence.replace(punctuation, '')
    word_tokens = word_tokenize(sentence)

        # Lemmatizing the verbs
    verb_lemmatized = [                  
        WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
        for word in word_tokens]

    # 2 - Lemmatizing the nouns
    noun_lemmatized = [                 
        WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
        for word in verb_lemmatized]
    
    #join_list = ' '.join(noun_lemmatized)
    
    return noun_lemmatized

In [20]:
filtered_review = df.iloc[:10000,:]

In [21]:
#Applying preprocessing on the both positive and negative reviews

clean_negative_review = []
cleaned_positive_review = []


for sentence in filtered_review.Negative_Review:
    clean_negative_review.append(preprocessing(sentence))

clean_negative_df = pd.DataFrame({'Review': clean_negative_review})    
clean_negative_df['Review'] = clean_negative_df['Review'].astype(str)


for sentence in filtered_review.Positive_Review:
    cleaned_positive_review.append(preprocessing(sentence))
    
clean_positive_df = pd.DataFrame({'Review': cleaned_positive_review})    
clean_positive_df['Review'] = clean_positive_df['Review'].astype(str)

clean_negative_df.shape,clean_positive_df.shape

((10000, 1), (10000, 1))

In [22]:
#Applying preprocessing on the both positive and negative reviews

clean_negative_review = []
cleaned_positive_review = []


for sentence in filtered_review.Negative_Review:
    clean_negative_review.append(preprocessing(sentence))

clean_negative_df = pd.DataFrame({'Review': clean_negative_review})    
clean_negative_df['Review'] = clean_negative_df['Review'].astype(str)


for sentence in filtered_review.Positive_Review:
    cleaned_positive_review.append(preprocessing(sentence))
    
clean_positive_df = pd.DataFrame({'Review': cleaned_positive_review})    
clean_positive_df['Review'] = clean_positive_df['Review'].astype(str)

clean_negative_df.shape,clean_positive_df.shape

((10000, 1), (10000, 1))

In [23]:
# Get the number of rows in the DataFrame
num_negative_rows = clean_negative_df.shape[0]
num_positive_rows = clean_positive_df.shape[0]

# Create a new column with all 0 and 1 values based on positive/ negative
clean_negative_df['Label'] = [0] * num_negative_rows
clean_positive_df['Label'] = [1] * num_positive_rows

clean_negative_df.head(2),clean_positive_df.head(2)
#Combine both dataframe together to form dataset
combined_df = pd.concat([clean_negative_df, clean_positive_df], axis=0, join='inner')

combined_df['Label'].value_counts()

Label
0    10000
1    10000
Name: count, dtype: int64

In [24]:
from sklearn.model_selection import train_test_split
X = combined_df['Review']
y = combined_df['Label']

X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True,test_size=0.3)

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vectorizer = CountVectorizer()

# Fit and transform the 'clean_text' column
features = vectorizer.fit_transform(combined_df.Review)

# Get the feature names
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame with the features as column names
X_bow = pd.DataFrame(features.toarray(), columns=feature_names)

In [26]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Feature/Target
X = X_bow
y = combined_df['Label']

# Pipeline vectorizer + Naive Bayes
pipeline_naive_bayes = make_pipeline(
    CountVectorizer(),
    MultinomialNB()
)

# Define the grid of parameters
parameters = {
    'countvectorizer__ngram_range': ((1,2), (3,4)),
    'multinomialnb__alpha': (0.1,1)
}

# Perform Grid Search
grid_search = GridSearchCV(
    pipeline_naive_bayes,
    parameters,
    scoring = "recall",
    cv = 5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(combined_df['Review'],combined_df['Label'])

# Best score
print(f"Best Score = {grid_search.best_score_}")

# Best params
print(f"Best params = {grid_search.best_params_}")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Score = 0.9323
Best params = {'countvectorizer__ngram_range': (1, 2), 'multinomialnb__alpha': 1}


In [69]:
naivebayes = MultinomialNB(alpha=1)

vectorizer = CountVectorizer(ngram_range=(1,2))

# Fit and transform the 'clean_text' column
features = vectorizer.fit_transform(combined_df.Review)

# Get the feature names
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame with the features as column names
X_bow = pd.DataFrame(features.toarray(), columns=feature_names)

cv_nb = cross_validate(
    naivebayes,
    X_bow,
    combined_df['Label'],
    scoring = "accuracy",
    cv=10
)

round(cv_nb['test_score'].mean(),2)

: 

: 

In [ ]:
Good_test = "Great room and the view was amazing. The buffet breakfast had so many options and so much of everything as well. \
    The shops under the hotel were really good and also lots of little eating places too.\
        Raffles is a few mins walk and you can walk to Clarke Quay as well.\
            Perfect location and would highly recommend."
            
Bad_test = "the view was horrible. breakfast had so little options and so little of everything. \
    The shops under the hotel were really terrible and very little eating places too."

In [55]:
pipeline_naive_bayes = make_pipeline(
    CountVectorizer(ngram_range=(1, 2)),
    MultinomialNB(alpha=1)
)


# Fit the pipeline on the training data
pipeline_naive_bayes.fit(combined_df['Review'], combined_df['Label'])

# Now the model is trained and ready for predictions

# Prepare the input text for prediction
input_text = "This is a great product!"

# Make predictions using the trained pipeline
predictions = pipeline_naive_bayes.predict_proba([Good_test])
prediction_label = pipeline_naive_bayes.predict([Good_test])

print(predictions,prediction_label)

[[1.12675014e-20 1.00000000e+00]] [1]


In [59]:
round(1.12675014e-20,12)

0.0